## Outline
1. Loading and cleaning
    * clustering ethnicity and aki
    * remove outliers
    * should I remove features with too many missing values?
2. Splitting into training and testing
3. Standardization
4. Imputation for regression and SVM
    * Median
    * KNN
    * MICE
5. Oversampling (skip)
6. Feature selection
    * L1 regularization
    * step forward / backward
    * genetic algorhthm
7. Model building
    * logistic regression
    * SVM
    * XGBoost
8. Evaluation: accuracy, precision, recall, f-measure, breakeven point, sensitivity, specificity, ROC

In [ ]:
! pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn import preprocessing
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.linear_model import LogisticRegression as logit
from sklearn.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import roc_curve, precision_recall_curve, RocCurveDisplay, PrecisionRecallDisplay
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Load & Clean data
* (drop column if more than 85% missing)
* map ethnicity to more general categories
* map AKI stage to yes/no
* Remove outliers

In [ ]:
df = pd.read_csv('sph6004_assignment1_data.csv')
df['aki'] = df['aki'].astype(object)
df.head()

In [ ]:
df_datatypes = df.dtypes
df_null_count = df.count()
df_null_precent = round((df.shape[0] - df_null_count) / df.shape[0] * 100, 2)
info = pd.DataFrame({'non-null count': df_null_count, 'Dtype': df_datatypes, 'null %': df_null_precent})
info

In [ ]:
info[info['null %'] > 80].index

In [ ]:
# drop if majority is missing
def drop_missing_col(df, info, threshold):
    missing_col = info[info['null %'] > threshold].index
    df.drop(columns=missing_col, axis=1, inplace=True)
    return df

df = drop_missing_col(df, info, 80)
df.shape

In [ ]:
# df['race'].unique()
df['race'].value_counts()

In [ ]:
def ethnicity_clustering(ethnicity_original):
    if ethnicity_original.startswith('ASIAN'):
        return 'ASIAN'
    elif ethnicity_original.startswith('WHITE'):
        return 'WHITE'
    elif ethnicity_original.startswith('HISPANIC'):
        return 'HISPANIC'
    elif ethnicity_original.startswith('BLACK'):
        return 'BLACK'
    elif ethnicity_original in ['UNKNOWN', 'UNABLE TO OBTAIN', 'PATIENT DECLINED TO ANSWER']:
        return 'UNKNOWN'
    else:
        return 'OTHERS'

def aki_clustering(aki):
    if aki == 0:
        return 0
    else:
        return 1

In [ ]:
df['ethnicity_new'] = df.loc[:, 'race'].apply(ethnicity_clustering)
print(df['ethnicity_new'].value_counts())
df.drop(columns=['race'], axis=1, inplace=True)
# X = pd.get_dummies(df, columns = ['ethnicity_new'], drop_first=False)

In [ ]:
df['gender'] = df['gender'].astype('category').cat.codes # 0: Female; 1: Male

In [ ]:
print(df['aki'].value_counts())
df['aki_binary'] = df.loc[:, 'aki'].apply(aki_clustering)
print(df['aki_binary'].value_counts())
df.drop(columns=['aki'], axis=1, inplace=True)

### Remove outliers using IQR because data are skewed

In [ ]:
df.describe()

In [ ]:
# df1=df.select_dtypes(exclude=['object'])
# for column in df1.iloc[:, 1:]:
#     plt.figure()
#     df.boxplot([column])

In [ ]:
df.iloc[:, 1:].hist(bins=15, figsize=(50, 30))

In [ ]:
def threshold(df, col):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    upper = q3 + 1.5*iqr
    lower = q1 - 1.5*iqr
    return upper, lower
def identify_outlier(df, col):
    upper, lower = threshold(df, col)
    print('upper and lower limits for column {}: '.format(col), upper, lower)
    res = (df[col] > upper) | (df[col] < lower)
    print('total records removed for column {}: '.format(col), sum(res))
    return res
def remove_outlier(df):
    cols = df.columns.to_list()
    cols.remove('id')
    cols.remove('aki_binary')
    cols.remove('gender')
    cols.remove('ethnicity_new')
    print(cols)
    for col in cols:
        outlier_flag = identify_outlier(df, col)
        df.loc[outlier_flag, col] = np.nan
    return df


In [ ]:
df_cleaned = remove_outlier(df)
del df
df_cleaned.head()


In [ ]:
df_cleaned.iloc[:, 1:].hist(bins=15, figsize=(50, 30))

In [ ]:
# drop columns with meaningless values (all 0s)
drop_cols = ['methemoglobin_min', 'methemoglobin_max', 'atyps_min', 'atyps_max', 'gcs_unable']
df_cleaned.drop(columns=drop_cols, axis=1, inplace=True)

## Splitting into training and testing sets

In [ ]:
y = df_cleaned['aki_binary']
X = df_cleaned.drop(columns=['id', 'aki_binary'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)
X_train.head()

## Standardization

In [ ]:
numeric_cols = list(X.columns)[0:-1]
sd = preprocessing.StandardScaler()
X_train[numeric_cols] = sd.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = sd.transform(X_test[numeric_cols])
X_train.head()

## Imputation for logistic regression and SVM
1. Imputation by median
2. MICE
3. KNN

In [ ]:
imp_list = [SimpleImputer(missing_values=np.nan, strategy='median'), KNNImputer(), IterativeImputer()]
def imp(imputer):
    train_imp = X_train.copy()
    test_imp = X_test.copy()
    print('copy done')
    train_imp.iloc[:,0:-1] = imputer.fit_transform(train_imp.iloc[:,0:-1])
    print('impute training done')
    test_imp.iloc[:,0:-1] = imputer.transform(test_imp.iloc[:,0:-1])
    print('impute testing done')
    return train_imp, test_imp

In [ ]:
imp_median = imp_list[0]
X_train_median_imp, X_test_median_imp = imp(imp_median)
# X_train_median_imp.to_csv('X_train_median_imp.csv')
# X_test_median_imp.to_csv('X_test_median_imp.csv')
# y_train.to_csv('y_train.csv')
# y_test.to_csv('y_test.csv')

In [ ]:
del X_train_knn_imp, X_test_knn_imp

In [ ]:
imp_knn = imp_list[1]
X_train_knn_imp, X_test_knn_imp = imp(imp_knn)

In [ ]:
X_train_knn_imp.to_csv('X_train_knn_imp.csv')
X_test_knn_imp.to_csv('X_test_knn_imp.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')

In [ ]:
X_train_knn_imp = pd.read_csv('X_train_knn_imp.csv', index_col=0)
X_test_knn_imp = pd.read_csv('X_test_knn_imp.csv', index_col=0)

In [ ]:
imp_mice = imp_list[2]
X_train_mice_imp, X_test_mice_imp = imp(imp_mice)

In [ ]:
X_train_mice_imp.to_csv('X_train_mice_imp.csv')
X_test_mice_imp.to_csv('X_test_mice_imp.csv')

In [ ]:
# one-hot encoding for ethnicity
X_train_median_imp = pd.get_dummies(X_train_median_imp, columns = ['ethnicity_new'], drop_first=False)
X_test_median_imp = pd.get_dummies(X_test_median_imp, columns = ['ethnicity_new'], drop_first=False)
# X_train_knn_imp = pd.get_dummies(X_train_knn_imp, columns = ['ethnicity_new'], drop_first=False)
# X_test_knn_imp = pd.get_dummies(X_test_knn_imp, columns = ['ethnicity_new'], drop_first=False)
# X_train_mice_imp = pd.get_dummies(X_train_mice_imp, columns = ['ethnicity_new'], drop_first=False)
# X_test_mice_imp = pd.get_dummies(X_test_mice_imp, columns = ['ethnicity_new'], drop_first=False)

## Feature Selection
1. L1 regularization for logistic regression
2. Step forward / backward
3. Genetic Algorithm

### L1 regularization of regression (elastic net)

In [ ]:
cv = StratifiedKFold(n_splits=3)
enet = LogisticRegression(penalty = 'elasticnet', solver='saga', max_iter=1e5)
enet_param_grid = {
                'C'     : np.arange(0.5,20,step=2),
                'l1_ratio'  :  np.arange(0,1,0.2)
            }
grid_search_enet = GridSearchCV(enet,
                           enet_param_grid,
                           scoring='roc_auc',
                           cv = cv,
                           return_train_score=True,
                           n_jobs = -1,
                           verbose=2)
grid_search_enet.fit(X_train_median_imp,y_train)

In [ ]:
print(grid_search_enet.best_estimator_)
grid_search_enet.best_score_

In [ ]:
y_pred = grid_search_enet.predict(X_test_median_imp)
y_pred_proba = grid_search_enet.predict_proba(X_test_median_imp)[::,1]
print('F1 score on test set: {:.4f}'.format(f1_score(y_test,y_pred)))
print('AUC score on test set: {:.4f}'.format(roc_auc_score(y_test,y_pred_proba)))
pd.crosstab(y_test,y_pred)

In [ ]:
model_enet = grid_search_enet.best_estimator_
model_enet.fit(X_train_median_imp, y_train)
y_pred_train = model_enet.predict(X_train_median_imp)
y_pred_proba_train = model_enet.predict_proba(X_train_median_imp)[::,1]
print('F1 score on training set: {:.4f}'.format(f1_score(y_train,y_pred_train)))
print('AUC score on training set: {:.4f}'.format(roc_auc_score(y_train,y_pred_proba_train)))
pd.crosstab(y_test,y_pred)

In [ ]:
model_enet = LogisticRegression(C=0.5, l1_ratio=0.8, max_iter=100000, penalty='elasticnet',
                   solver='saga')
model_enet.fit(X_train_median_imp, y_train)

y_pred_train = model_enet.predict(X_train_median_imp)
y_pred_proba_train = model_enet.predict_proba(X_train_median_imp)[::,1]
print('F1 score on training set: {:.4f}'.format(f1_score(y_train,y_pred_train)))
print('AUC score on training set: {:.4f}'.format(roc_auc_score(y_train,y_pred_proba_train)))
print('Accuracy score on training set: {:.4f}'.format(accuracy_score(y_train,y_pred_train)))

y_pred = model_enet.predict(X_test_median_imp)
y_pred_proba = model_enet.predict_proba(X_test_median_imp)[::,1]
print('\nF1 score on test set: {:.4f}'.format(f1_score(y_test,y_pred)))
print('AUC score on test set: {:.4f}'.format(roc_auc_score(y_test,y_pred_proba)))
print('Accuracy score on test set: {:.4f}'.format(accuracy_score(y_test,y_pred)))

In [ ]:

auc_plot = RocCurveDisplay.from_estimator(grid_search_enet, X_test_median_imp, y_test)

# precision, recall, _ = precision_recall_curve(y_test, y_pred)
pr_plot = PrecisionRecallDisplay.from_estimator(grid_search_enet, X_test_median_imp, y_test)
plt.show()

In [ ]:
enet_coef = pd.DataFrame({'column names': X_train_median_imp.columns, 'coef': model_enet.coef_[0]})
enet_coef.sort_values(by=['coef'], ascending=False, inplace=True, key=abs)
enet_coef

In [ ]:
coef_plot = enet_coef.boxplot()
plt.title('Coefficient Distribution of Elastic Net Model')

### Genetic regression

Logistic regression

In [ ]:
! pip install sklearn_genetic

In [ ]:
from genetic_selection import GeneticSelectionCV

estimator = LogisticRegression(max_iter=int(1e5))

selector = GeneticSelectionCV(
    estimator,
    cv=3,
    verbose=1,
    scoring="roc_auc",
    max_features=100,
    n_population=50,
    crossover_proba=0.5,
    mutation_proba=0.2,
    n_generations=50,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3,
    n_gen_no_change=10,
    caching=True,
    n_jobs=-1,
)
selector = selector.fit(X_train_median_imp, y_train)

In [ ]:
print(selector.support_)
print(selector.n_features_)
print(selector.generation_scores_)

In [ ]:
print('train AUC:', selector.score(X_train_median_imp, y_train))
print('test AUC:', selector.score(X_test_median_imp, y_test))

In [ ]:
pd.DataFrame(X_train_median_imp.columns[selector.support_])

In [ ]:
X_train_median_imp.columns[selector.support_]

In [ ]:
# selected_cols = X_train_median_imp.columns[selector.support_]
# selected_cols = ['admission_age', 'heart_rate_max', 'sbp_min', 'sbp_max', 'sbp_mean',
#        'dbp_min', 'mbp_min', 'mbp_max', 'mbp_mean', 'resp_rate_max',
#        'resp_rate_mean', 'temperature_min', 'temperature_max', 'spo2_min',
#        'spo2_mean', 'glucose_max', 'lactate_max', 'aado2_calc_max',
#        'pao2fio2ratio_min', 'baseexcess_min', 'baseexcess_max',
#        'hemoglobin_max', 'calcium_min', 'calcium_max', 'hematocrit_max.1',
#        'hemoglobin_min.1', 'albumin_min', 'aniongap_min', 'aniongap_max',
#        'bicarbonate_min.1', 'bicarbonate_max.1', 'bun_min', 'bun_max',
#        'calcium_max.1', 'chloride_max.1', 'glucose_max.2', 'potassium_max.1',
#        'abs_lymphocytes_min', 'fibrinogen_max', 'inr_max', 'pt_min', 'ptt_max',
#        'alp_min', 'bilirubin_total_min', 'ld_ldh_min', 'gcs_motor',
#        'gcs_verbal', 'gcs_eyes', 'weight_admit', 'ethnicity_new_ASIAN']
selected_cols = X_train_median_imp.columns[selector.support_].to_list()
X_train_median_imp_selected = X_train_median_imp[selected_cols]
X_test_median_imp_selected = X_test_median_imp[selected_cols]

selected_cols2 = selected_cols[:-1] + ['ethnicity_new']

# X_train_selected = X_train[selected_cols2]
# X_test_selected = X_test[selected_cols2]

X_train_median_imp_selected.head()

## Model Building & tuning
1. Logistic regression
2. Non-linear SVM
3. XGBoost

In [ ]:
# TODO: training F1 score, AUC plot for test set
from xgboost import XGBClassifier as XGBC
models = {
    'Logistic_Regression': LogisticRegression(max_iter=int(1e5)),
    'SVM_rbf':SVC(kernel='rbf', probability=True),
    'XGBoost':XGBC()
}

stratifiedCV = StratifiedKFold(n_splits=3)


params = {
    'Logistic_Regression':{
        'C':np.arange(0.5,20,step=2)
    },
    'XGBoost':{
        'n_estimators':np.arange(start=2,stop=500,step=30),
        'max_depth':np.arange(start=2,stop=6)
    },
    'SVM_rbf':{
        'C':np.arange(0.5,5,step=1)
    }
}

records = {}

for model in models:
    print(model)
    BestParams = GridSearchCV(
        models[model],
        param_grid = params[model],
        scoring='roc_auc',
        cv=stratifiedCV,
        n_jobs=-1,
        verbose=1
    )

    BestParams.fit(X_train_median_imp_selected,y_train)
    print(BestParams.best_estimator_)
    # Xy_test[model] = BestParams.predict(X_test_median_imp).astype(str)
    records[model] = BestParams
    print('For {} cross validation AUC score is {:.4f}'.format(model,BestParams.best_score_))

    y_pred_train = BestParams.predict(X_train_median_imp_selected)
    y_pred_proba_train = BestParams.predict_proba(X_train_median_imp_selected)[::,1]
    print('F1 score on training set: {:.4f}'.format(f1_score(y_train,y_pred_train)))
    print('AUC score on training set: {:.4f}'.format(roc_auc_score(y_train,y_pred_proba_train)))
    print('Accuracy score on training set: {:.4f}'.format(accuracy_score(y_train,y_pred_train)))
    print(pd.crosstab(y_train,y_pred_train))

    y_pred = BestParams.predict(X_test_median_imp_selected)
    y_pred_proba = BestParams.predict_proba(X_test_median_imp_selected)[::,1]
    print('F1 score on test set: {:.4f}'.format(f1_score(y_test,y_pred)))
    print('AUC score on test set: {:.4f}'.format(roc_auc_score(y_test,y_pred_proba)))
    print('Accuracy score on test set: {:.4f}'.format(accuracy_score(y_train,y_pred_train)))
    print(pd.crosstab(y_test,y_pred))

    auc_plot = RocCurveDisplay.from_estimator(BestParams, X_test_median_imp_selected, y_test)
    pr_plot = PrecisionRecallDisplay.from_estimator(BestParams, X_test_median_imp_selected, y_test)
    plt.show()

In [ ]:
from IPython.display import clear_output

clear_output()